<a href="https://colab.research.google.com/github/MeryemLaouinate/Classification-smily-neutral-faces/blob/main/expression_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import cv2
import glob,os
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
print ("tensorflow version",tf.__version__)
%matplotlib inline

tensorflow version 2.4.0


In [2]:
# reading images from the data folder
# label, smile = 1, neutral = 0
(img_width, img_height) = (45, 25)
def load_images_from_folder(folder):
    (images, lables, names, id) = ([], [], {}, 0)
    for (subdirs, dirs, files) in os.walk(folder):
        print (subdirs, dirs)
        for subdir in dirs:
            names[id] = subdir
            subjectpath = os.path.join(folder, subdir)
            for filename in os.listdir(subjectpath):
                path = subjectpath + '/' + filename
                lable = id
                img = cv2.imread(path, 0) 
                img = cv2.resize(img,(img_width, img_height)) 
                images.append(img)
                lables.append(int(lable))
            id += 1
        print(names)
        print(lables)
        return images, lables, names

In [6]:
X,Y,classes = load_images_from_folder("/content/sample_data/Data")

(X,Y) = [np.array(lis) for lis in [X, Y]]
Y = pd.get_dummies(Y) #converting labels to one-hot, Used Pandas for it.

/content/sample_data/Data ['.ipynb_checkpoints', 'neutral', 'smile']
{0: '.ipynb_checkpoints', 1: 'neutral', 2: 'smile'}
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [7]:
print ("Height of the image:  " + str(X.shape[1]))
print ("Width of the image:   "  + str(X.shape[2]))
print ("Total number of data: "+ str(len(X)))

Height of the image:  25
Width of the image:   45
Total number of data: 107


In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state=42)

In [9]:

X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state=42)
print ("lenght of train data: "+str(len(X_train))) 
print ("lenght of test data:  "+str(len(X_test)))

lenght of train data: 71
lenght of test data:  36


In [10]:
train_set_x_flatten = X_train.reshape(X_train.shape[0],-1)
test_set_x_flatten = X_test.reshape(X_test.shape[0],-1)
print ("train_set_x_flatten: "+str(train_set_x_flatten.shape))
print ("test_set_x_flatten : "+str(test_set_x_flatten.shape))

print (train_set_x_flatten)

train_set_x_flatten: (71, 1125)
test_set_x_flatten : (36, 1125)
[[ 99 110 109 ... 134 131 124]
 [131 134 136 ... 123 122 117]
 [163 165 166 ... 161 160 155]
 ...
 [124 123 124 ... 192 200 204]
 [173 181 188 ... 166 163 159]
 [158 145 156 ... 150 140 142]]


In [11]:
#Normalizing data, conveting all pixel value in range between 0-1
train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255.
print ("length of train set: " + str(len(train_set_x)))
print ("length of test set: " + str(len(test_set_x)))
print ("shape of train set: "+ str(train_set_x.shape))
print ("shape of test set: "+ str(test_set_x.shape))

length of train set: 71
length of test set: 36
shape of train set: (71, 1125)
shape of test set: (36, 1125)


In [14]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [39]:
x = tf.placeholder(tf.float32, [None, 1125])
W = tf.Variable(tf.zeros([1125, 2]), dtype=tf.float32, name="w1")
b = tf.Variable(tf.zeros([2]),  dtype=tf.float32, name="bias")
y = tf.nn.softmax(tf.matmul(x, W) + b, name="first_operation")
y_ = tf.placeholder(tf.float32, [None, 2], name = "final_output")
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [29]:
learning_rate = 0.1
print ("training with learning rate: " + str(learning_rate))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(15000):
        trainStep,loss = sess.run([train_step, cross_entropy], feed_dict={x: train_set_x, y_: y_train})
        if step%1000==0:
            print ("losses after per 1000 iteration: ",loss)

    save_path = saver.save(sess, "./model/emotion_model")

    print("Model saved in file: %s" % save_path)

    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("accuracy with learning rate: " ,str(learning_rate), sess.run(accuracy, feed_dict={x:test_set_x , y_: y_test}))

training with learning rate: 0.1
losses after per 1000 iteration:  0.6931472
losses after per 1000 iteration:  0.011277198
losses after per 1000 iteration:  0.0065985005
losses after per 1000 iteration:  0.004720176
losses after per 1000 iteration:  0.0036882658
losses after per 1000 iteration:  0.0030317826
losses after per 1000 iteration:  0.0025760354
losses after per 1000 iteration:  0.0022406802
losses after per 1000 iteration:  0.0019833003
losses after per 1000 iteration:  0.0017794182
losses after per 1000 iteration:  0.0016138569
losses after per 1000 iteration:  0.001476693
losses after per 1000 iteration:  0.001361178
losses after per 1000 iteration:  0.0012625192
losses after per 1000 iteration:  0.0011773005
Model saved in file: ./model/emotion_model
accuracy with learning rate:  0.1 0.9722222


In [44]:
init_op = tf.global_variables_initializer() 

In [54]:

saver = tf.train.Saver()
with tf.Session() as sess:
    #saver.restore(sess, "/content/model/emotion_model")
    print("w1:", sess.run(init_op))
    #print("bias:", sess.run(b))
    img = cv2.imread("/content/sample_data/test/test1.jpg", 0) 
    img = cv2.resize(img,(img_width, img_height)) # Resizing all the images
    image=[]
    image.append(img)
    image_test = np.array(image)
    test_image_flatten = image_test.reshape(image_test.shape[0],-1)
    test_image_normalized = test_image_flatten/255.
    #print (test_image_normalized.shape)
    x_ = tf.cast(test_image_normalized, tf.float32)
    y = tf.nn.softmax(tf.matmul(x_, W) + b)
    #print (sess.run(y))
    indx = sess.run(tf.argmax(y,1))
    if indx==1:
        print ("smile")
    elif indx==0:
        print ("neutral")

w1: None
neutral
